In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
import datetime

In [2]:
my_dia = pd.read_csv("C:/Users/Caroline Kries/sciebo/Forschungsprojekt Medikamenteninteraktion/03_Daten/my_dia")
my_medi = pd.read_csv("C:/Users/Caroline Kries/sciebo/Forschungsprojekt Medikamenteninteraktion/03_Daten/my_medi")
my_inhalt = pd.read_csv("C:/Users/Caroline Kries/sciebo/Forschungsprojekt Medikamenteninteraktion/03_Daten/my_inhalt")


my_dia["giltab"] = pd.to_datetime(my_dia.giltab)
pzn_wirk = my_inhalt[["pzn","bezeichnung"]]
pzn_wirk = pzn_wirk.rename(columns={"pzn":"pzn","bezeichnung":"wirkstoff"})
my_medi_wirk = pd.merge(my_medi, pzn_wirk,how="left",on="pzn")
my_medi_wirk["tag"] = pd.to_datetime(my_medi_wirk["tag"])

klienten = my_dia.klientid.unique()


C:\Users\Caroline Kries\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
my_medi_wirk

,Unnamed: 0,id,klientid,tag,datum,bezeichnung,pzn,menge,einheit,wirkstoff
0,38851,38852,3734,2007-08-18,2007-08-18 06:30:00,Euthyrox 75 Mikrogramm TAB,2754708,1,Stck,Levothyroxin natrium
1,38852,38853,3734,2007-08-19,2007-08-19 06:30:00,Euthyrox 75 Mikrogramm TAB,2754708,1,Stck,Levothyroxin natrium
2,38853,38854,3734,2007-08-20,2007-08-20 06:30:00,Euthyrox 75 Mikrogramm TAB,2754708,1,Stck,Levothyroxin natrium
3,38854,38855,3734,2007-08-21,2007-08-21 06:30:00,Euthyrox 75 Mikrogramm TAB,2754708,1,Stck,Levothyroxin natrium
4,38855,38856,3734,2007-08-22,2007-08-22 06:30:00,Euthyrox 75 Mikrogramm TAB,2754708,1,Stck,Levothyroxin natrium
...,...,...,...,...,...,...,...,...,...,...
1314188,26335959,26335960,22906,2021-11-08,2021-11-08 18:00:00,"Eliquis 2,5mg FTA",8400029.0,1,Stck,Apixaban
1314189,26335960,26335961,22906,2021-11-09,2021-11-09 06:30:00,"Eliquis 2,5mg FTA",8400029.0,1,Stck,Apixaban
1314190,26335961,26335962,22906,2021-11-09,2021-11-09 18:00:00,"Eliquis 2,5mg FTA",8400029.0,1,Stck,Apixaban
1314191,26335962,26335963,22906,2021-11-10,2021-11-10 06:30:00,"Eliquis 2,5mg FTA",8400029.0,1,Stck,Apixaban


[4, 4, 4]

In [16]:
for klient in klienten:

    medi1 = my_medi_wirk[my_medi_wirk.klientid == klient]
    df = medi1[["tag", "wirkstoff"]]
    df.dropna(how="any", inplace=True)

    names = list(df.wirkstoff)
    uniquenames = list(set(names))
    dates = list(df.tag)
    zip_list = list(zip(names, dates))

    #mydict = {"Medikament":[Timestamp1, Timestamp2, ...],...}
    mydict = {}
    for x in uniquenames:
        mydict[x] = []
        for z in zip_list:
            if z[0] == x:
                mydict[x].append(z[1])

    #begin = {"Medikament":[Timestamp1, Timestamp2, ...],...}
    # Timestamps geben ersten bzw. letzten Tag der Medikamentation an
    begin = {}
    end = {}
    for name in mydict.keys():
        newtime = [mydict[name][0]]
        endtime = []
        currenttime = mydict[name][0]
        for timestamp in mydict[name]:
            if timestamp - currenttime > datetime.timedelta(days=14):
                newtime.append(timestamp)
                endtime.append(currenttime)
            currenttime = timestamp
        endtime.append(mydict[name][-1])
        begin[name] = newtime
        end[name] = endtime

    #names und dates für die plot_timeline Methode mit je einem Zeitstempel für jedes Medikament
    names = []
    begin_dates = []
    end_dates = []
    for name in list(begin.keys()):
        timelist = begin[name]
        for time in timelist:
            names.append(name)
            begin_dates.append(time)
        timelist2 = end[name]
        for time in timelist2:
            end_dates.append(time)

    klientid = [klient] * len(names)
    dict = {"klientid": klientid, "tag_beginn": begin_dates, "tag_ende": end_dates, "wirkstoff": names}

    if klient == klienten[0]:
        current_df = pd.DataFrame(dict)
    else:
        new_df = pd.DataFrame(dict)
        current_df = pd.concat([current_df, new_df])

    drugs = list(set(names))
    klientid = [klient] * len(drugs)
    dict2 = {"klientid": klientid, "wirkstoff": drugs}

    if klient == klienten[0]:
        only_drugs = pd.DataFrame(dict2)
    else:
        new_only_drugs = pd.DataFrame(dict2)
        only_drugs = pd.concat([only_drugs, new_only_drugs])

<ipython-input-16-c5682b40d9aa>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(how="any", inplace=True)
<ipython-input-16-c5682b40d9aa>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(how="any", inplace=True)
<ipython-input-16-c5682b40d9aa>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(how="any", inplace=True)
<ipython-input-16-c5682b40d9aa>:5: SettingWithCopyWarning: 
A value is trying to be set on 

In [17]:
only_drugs

,klientid,wirkstoff
0,3181.0,Kombi-Präparat
1,3181.0,Macrogol 3350
2,3181.0,Furosemid
3,3181.0,Dexamethason
4,3181.0,Omeprazol
...,...,...
27,22906.0,Kaliumchlorid
28,22906.0,Metamizol natrium-1-Wasser
29,22906.0,Levetiracetam
30,22906.0,Goserelin acetat (1:x)


In [22]:
counts = current_df.wirkstoff.value_counts()

In [23]:
counts[:80]

Metamizol natrium-1-Wasser    126
Insulin, normal (human)       123
Insulin-Isophan (human)       107
Torasemid                      85
Acetylsalicylsäure             72
                             ... 
Pipamperon dihydrochlorid      10
Tamsulosin hydrochlorid         9
Captopril                       9
Olmesartan medoxomil            9
Glibenclamid                    9
Name: wirkstoff, Length: 80, dtype: int64

In [24]:
counts = only_drugs.wirkstoff.value_counts()
counts

Metamizol natrium-1-Wasser        39
Torasemid                         37
Insulin, normal (human)           34
Pantoprazol natrium-1,5-Wasser    30
Insulin-Isophan (human)           29
                                  ..
Hydrocortison                      1
Anamirta cocculus                  1
Docusat natrium                    1
Flurazepam                         1
Pfefferminzöl                      1
Name: wirkstoff, Length: 393, dtype: int64

In [29]:
binarytable = only_drugs.groupby(['klientid', 'wirkstoff']).size().unstack(fill_value=0)

In [30]:
binarytable.sum()

wirkstoff
Acetylcystein                     12
Acetylsalicylsäure                27
Aciclovir                          2
Alendronsäure natrium              1
Alendronsäure natrium-1-Wasser     1
                                  ..
Zinkoxid                           4
Zitronenöl-Destillat               1
Zolpidem hemitartrat               1
Zopiclon                           7
beta-Acetyldigoxin                 1
Length: 393, dtype: int64